<a href="https://colab.research.google.com/github/Ekaagra08/Resume-Parsing-NER-Rule_based-/blob/main/Custom_Resume_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==2.1.4

     |████████████████████████████████| 29.8 MB 16.1 MB/s 
     |████████████████████████████████| 3.2 MB 51.4 MB/s 
     |████████████████████████████████| 2.1 MB 40.3 MB/s 
     |████████████████████████████████| 82 kB 435 kB/s 
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
ERROR: pip's dependency resol

In [2]:
import json
import re
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score


In [3]:
import spacy
from spacy import displacy
!python -m spacy download en_core_web_md
import en_core_web_md

     |████████████████████████████████| 95.4 MB 1.3 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-py3-none-any.whl size=97126236 sha256=679bf6d6758ad33e0b77f126afcfb85c5b28b5c0a195b371fdd7cad687f82837
  Stored in directory: /tmp/pip-ephem-wheel-cache-t5b8_j2y/wheels/1a/4e/53/ca2bd8efb94658d2425a0a2d998ffddc2db7ea1378421f8565
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [13]:
def convert_json_for_spacy(FilePath):
    try:
      converted_data = []
      lines=[]
      with open(FilePath, 'r') as f:
          lines = f.readlines()

      for line in lines:
          data = json.loads(line)
          text = data['content'].replace("\n", " ")
          entities = []
          data_annotations = data['annotation']
          if data_annotations is not None:
              for annotation in data_annotations:
                  #only a single point in text annotation.
                  point = annotation['points'][0]
                  labels = annotation['label']
                  # handle both list of labels or a single label.
                  if not isinstance(labels, list):
                      labels = [labels]

                  for label in labels:
                      point_start = point['start']
                      point_end = point['end']
                      point_text = point['text']

                      lstrip_diff = len(point_text) - len(point_text.lstrip())
                      rstrip_diff = len(point_text) - len(point_text.rstrip())
                      if lstrip_diff != 0:
                          point_start = point_start + lstrip_diff
                      if rstrip_diff != 0:
                          point_end = point_end - rstrip_diff
                      entities.append((point_start, point_end + 1 , label))
          
          converted_data.append((text, {"entities" : entities}))
      return converted_data
      
    except Exception as e:
      logging.exception("Unable to process " + FilePath + "\n" + "error = " + str(e))
      return None    

################################################################################################

def trim_entity_spans(data: list) -> list:
# removes extra white spaces from entity span to prevent overlaping
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    
    return cleaned_data


In [19]:
train_data_clean = trim_entity_spans(convert_json_for_spacy("/content/drive/MyDrive/Resume Parsing (NER + Rule-based)/traindata.json"))
print(train_data_clean[0])

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [22]:
def train_spacyNER():

  # creating blank eng-language class and add the built-in pipeline components to the pipeline
  c_nlp = spacy.blank("en")
  if 'ner' not in c_nlp.pipe_names:
    ner = c_nlp.create_pipe('ner')
    c_nlp.add_pipe(ner,last = True)

  # adding custome lables from resume
  for _, annotation in train_data_clean:
    for ent in annotation.get('entities'):
      ner.add_label(ent[2])    

  # other pipes to disabled during training
  other_pipes = [pipe for pipe in c_nlp.pipe_names if pipe != 'ner']
  with c_nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = c_nlp.begin_training()
    for itn in range(10):
      print("Statring iteration " + str(itn + 1))
      random.shuffle(train_data_clean)
      losses = {}
      for text, annotations in train_data_clean:
        c_nlp.update(
            [text],  # batch of texts
            [annotations],  # batch of annotations
            drop=0.2,  # dropout - make it harder to memorise data
            sgd=optimizer,  # callable to update weights
            losses=losses)
      print(losses)
  return c_nlp   

In [23]:
nlp = train_spacyNER()

Statring iteration 1
{'ner': 25523.626846685635}
Statring iteration 2
{'ner': 20646.36639341638}
Statring iteration 3
{'ner': 17600.827732211812}
Statring iteration 4
{'ner': 14516.906896550738}
Statring iteration 5
{'ner': 10964.033806758005}
Statring iteration 6
{'ner': 11535.556045774105}
Statring iteration 7
{'ner': 10664.032842613178}
Statring iteration 8
{'ner': 8828.792679796434}
Statring iteration 9
{'ner': 9942.307063860306}
Statring iteration 10
{'ner': 7819.600675078184}


In [33]:
doc=nlp(train_data_clean[0][0])
for ent in doc.ents:
  print(f'{ent.label_.upper():{25}}- {ent.text}')

NAME                     - Tapan kumar Nayak
LOCATION                 - Bhubaneshwar
EMAIL ADDRESS            - indeed.com/r/Tapan-kumar-Nayak/ da1f5ffb3c4c4b17
YEARS OF EXPERIENCE      - 10years
DESIGNATION              - G4S Security
COMPANIES WORKED AT      - Infosys
LOCATION                 - Bhubaneshwar


In [35]:
#test the model and evaluate it
examples = convert_json_for_spacy("/content/drive/MyDrive/Resume Parsing (NER + Rule-based)/testdata.json")
tp=0
tr=0
tf=0

ta=0
c=0        
for text,annot in examples:

    f=open("resume"+str(c)+".txt","w")
    doc_to_test=nlp(text)
    d={}
    for ent in doc_to_test.ents:
        d[ent.label_]=[]
    for ent in doc_to_test.ents:
        d[ent.label_].append(ent.text)

    for i in set(d.keys()):

        f.write("\n\n")
        f.write(i +":"+"\n")
        for j in set(d[i]):
            f.write(j.replace('\n','')+"\n")
    d={}
    for ent in doc_to_test.ents:
        d[ent.label_]=[0,0,0,0,0,0]
    for ent in doc_to_test.ents:
        doc_gold_text= nlp.make_doc(text)
        gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
        y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
        y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
        if(d[ent.label_][0]==0):
            #f.write("For Entity "+ent.label_+"\n")   
            #f.write(classification_report(y_true, y_pred)+"\n")
            (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
            a=accuracy_score(y_true,y_pred)
            d[ent.label_][0]=1
            d[ent.label_][1]+=p
            d[ent.label_][2]+=r
            d[ent.label_][3]+=f
            d[ent.label_][4]+=a
            d[ent.label_][5]+=1
    c+=1
for i in d:
    print("\n For Entity "+i+"\n")
    print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
    print("Precision : "+str(d[i][1]/d[i][5]))
    print("Recall : "+str(d[i][2]/d[i][5]))
    print("F-score : "+str(d[i][3]/d[i][5]))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMe


 For Entity Name

Accuracy : 99.82683982683983%
Precision : 0.9982714019140732
Recall : 0.9982683982683983
F-score : 0.9979805495430495

 For Entity Location

Accuracy : 99.3073593073593%
Precision : 0.9931216097593915
Recall : 0.9930735930735931
F-score : 0.9903150762281195

 For Entity Email Address

Accuracy : 99.48051948051948%
Precision : 1.0
Recall : 0.9948051948051948
F-score : 0.9973958333333334

 For Entity Designation

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0

 For Entity Companies worked at

Accuracy : 99.13419913419914%
Precision : 0.9914170174135513
Recall : 0.9913419913419913
F-score : 0.9877533258734302

 For Entity College Name

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0

 For Entity Graduation Year

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0

 For Entity Skills

Accuracy : 100.0%
Precision : 1.0
Recall : 1.0
F-score : 1.0
